In [28]:
from copy import copy
import queue
import random
import numpy as np

In [35]:
class Board:

    def __init__(self, N=4):
        self.N = N
        self.board = np.arange(N**2).reshape((N, N)).tolist()
        self.board[-1][-1] = '*'
        self.goal = []
        for i in self.board:
            self.goal.append(tuple(i))
        self.goal = tuple(self.goal)  # Unaltered puzzle, already in Goal-state.
        self.empty = [N-1, N-1]

    def __repr__(self):
        """ Format puzzle board in neat and understandable format."""
        string = ''
        for row in self.board:
            for num in row:
                if len(str(num)) == 1:
                    string += '   ' + str(num)
                elif len(str(num)) > 1:
                    string += '  ' + str(num)
            string += '\n'
        return string

    def convert_to_tuple(self, ar):
        result = []
        for i in ar:
            result.append(tuple(i))
        return tuple(result)

    def convert_to_list(self, tup):
        result = []
        for i in tup:
            result.append(list(i))
        return result

    def match(self, copy):  # Locate empty spot in puzzle.
        a = Board(self.N)  # Create new Board object.
        a.board = copy  # Set board attribute to `copy`.
        for row in range(0, len(a.board)):
            for col in range(0, len(a.board[0])):
                if a.board[row][col] == '*':  # Search for asterisk.
                    a.empty = [row, col]  # When found, store the location of empty spot as attribute.
        result = []
        for i in a.board:
            result.append(list(i))
        a.board = result  # Convert copy of board to list format.
        return a

    def move_up(self): # move empty block up
        try:
            if self.empty[0] != 0:
                tmp = self.board[self.empty[0]-1][self.empty[1]]
                self.board[self.empty[0]-1][self.empty[1]] = '*'
                self.board[self.empty[0]][self.empty[1]] = tmp
                self.empty = [self.empty[0]-1, self.empty[1]]
        except IndexError:
            pass

    def move_down(self): # move empty block down
        try:
            tmp = self.board[self.empty[0]+1][self.empty[1]]
            self.board[self.empty[0]+1][self.empty[1]] = '*'
            self.board[self.empty[0]][self.empty[1]] = tmp
            self.empty = [self.empty[0]+1, self.empty[1]]
        except IndexError:
            pass

    def move_right(self): # move empty block right
        try:
            tmp = self.board[self.empty[0]][self.empty[1]+1]
            self.board[self.empty[0]][self.empty[1]+1] = '*'
            self.board[self.empty[0]][self.empty[1]] = tmp
            self.empty = [self.empty[0], self.empty[1]+1]
        except IndexError:
            pass

    def move_left(self): # move empty block left
        try:
            if self.empty[1] != 0:
                tmp = self.board[self.empty[0]][self.empty[1]-1]
                self.board[self.empty[0]][self.empty[1]-1] = '*'
                self.board[self.empty[0]][self.empty[1]] = tmp
                self.empty = [self.empty[0], self.empty[1]-1]
        except IndexError:
            pass

    def shuffle(self, steps):
        for i in range(0, steps):
            direction = random.randrange(1, 5)
            if direction == 1:
                self.move_up()
            elif direction == 2:
                self.move_right()
            elif direction == 3:
                self.move_left()
            elif direction == 4:
                self.move_down()

    def solve_bfs(self):
        start = self.convert_to_tuple(self.board)
        pred = {}
        visited = []
        frontier = queue.Queue()
        frontier.put(start)
        
        while frontier.qsize() > 0:
            # print(F"Size of Frontier: {frontier.qsize()}")
            tmp = frontier.get()
            
            if tmp == self.goal:  # Check if the puzzle is solved.
                path = []
                while tmp != start:
                    path.append(pred[tmp][1])
                    tmp = pred[tmp][0]
                return path[::-1]
            
            if tmp not in visited:  # Otherwise, check if the puzzle is in a novel state.
                # If puzzle is in novel state, expand all possible movements as new puzzles.
                visited.append(tmp)  # Store a copy of tmp in visited list.
                tmpboard = self.match(tmp)  # Generate temporary board from tmp.
                tmpboard.move_up()  # Move empty spot up,
                if self.convert_to_tuple(tmpboard.board) != tmp:  # Check if it is unchanged.
                    frontier.put(self.convert_to_tuple(tmpboard.board))  # If it is changed, push to frontier.
                    if self.convert_to_tuple(tmpboard.board) not in pred:  # Check if board already has a predicted movement,
                        pred[self.convert_to_tuple(tmpboard.board)]=[tmp, 'up']  # If it does not, add it to predictions.

                
                tmpboard = self.match(tmp)
                tmpboard.move_down()
                if self.convert_to_tuple(tmpboard.board) != tmp:
                    frontier.put(self.convert_to_tuple(tmpboard.board))
                    if self.convert_to_tuple(tmpboard.board) not in pred:
                        pred[self.convert_to_tuple(tmpboard.board)]=[tmp, 'down']

                        
                tmpboard = self.match(tmp)
                tmpboard.move_right()
                if self.convert_to_tuple(tmpboard.board) != tmp:
                    frontier.put(self.convert_to_tuple(tmpboard.board))
                    if self.convert_to_tuple(tmpboard.board) not in pred:
                        pred[self.convert_to_tuple(tmpboard.board)]=[tmp, 'right']

                
                tmpboard = self.match(tmp)
                tmpboard.move_left()
                if self.convert_to_tuple(tmpboard.board) != tmp:
                    frontier.put(self.convert_to_tuple(tmpboard.board))
                    if self.convert_to_tuple(tmpboard.board) not in pred:
                        pred[self.convert_to_tuple(tmpboard.board)]=[tmp, 'left']

        raise Exception('There is no solution.')

    def solve_dfs(self):
        start = self.convert_to_tuple(self.board)
        pred = {}
        visited = []
        frontier = queue.LifoQueue()
        frontier.put(start)
        
        while frontier.qsize() > 0:
            # print(F"Size of Frontier: {frontier.qsize()}")
            tmp = frontier.get()
            
            if tmp == self.goal:  # Check if the puzzle is solved.
                path = []
                while tmp != start:
                    path.append(pred[tmp][1])
                    tmp = pred[tmp][0]
                return path[::-1]
            
            if tmp not in visited:  # Otherwise, check if the puzzle is in a novel state.
                # If puzzle is in novel state, expand all possible movements as new puzzles.
                visited.append(tmp)  # Store a copy of tmp in visited list.
                tmpboard = self.match(tmp)  # Generate temporary board from tmp.
                tmpboard.move_up()  # Move empty spot up,
                if self.convert_to_tuple(tmpboard.board) != tmp:  # Check if it is unchanged.
                    frontier.put(self.convert_to_tuple(tmpboard.board))  # If it is changed, push to frontier.
                    if self.convert_to_tuple(tmpboard.board) not in pred:  # Check if board already has a predicted movement,
                        pred[self.convert_to_tuple(tmpboard.board)]=[tmp, 'up']  # If it does not, add it to predictions.

                
                tmpboard = self.match(tmp)
                tmpboard.move_down()
                if self.convert_to_tuple(tmpboard.board) != tmp:
                    frontier.put(self.convert_to_tuple(tmpboard.board))
                    if self.convert_to_tuple(tmpboard.board) not in pred:
                        pred[self.convert_to_tuple(tmpboard.board)]=[tmp, 'down']

                        
                tmpboard = self.match(tmp)
                tmpboard.move_right()
                if self.convert_to_tuple(tmpboard.board) != tmp:
                    frontier.put(self.convert_to_tuple(tmpboard.board))
                    if self.convert_to_tuple(tmpboard.board) not in pred:
                        pred[self.convert_to_tuple(tmpboard.board)]=[tmp, 'right']

                
                tmpboard = self.match(tmp)
                tmpboard.move_left()
                if self.convert_to_tuple(tmpboard.board) != tmp:
                    frontier.put(self.convert_to_tuple(tmpboard.board))
                    if self.convert_to_tuple(tmpboard.board) not in pred:
                        pred[self.convert_to_tuple(tmpboard.board)]=[tmp, 'left']

        raise Exception('There is no solution.')

In [50]:
N = 5
board = Board(N)  # Initialize board of size NxN. 
print("Unaltered Board\n", board)
board.shuffle(3)
print ("Shuffled Board\n", board)

path = board.solve_bfs()
print(F"BFS Path: {path}")
path = board.solve_dfs()
print (F"DFS Path: {path}")

for dir in path:
    if dir == 'right':
        board.move_right()
    if dir == 'left': 
        board.move_left()
    if dir == 'up':
        board.move_up()
    if dir == 'down':
        board.move_down()

print ("Solved Board\n", board)

Unaltered Board
    0   1   2   3   4
   5   6   7   8   9
  10  11  12  13  14
  15  16  17  18  19
  20  21  22  23   *

Shuffled Board
    0   1   2   3   4
   5   6   7   8   9
  10  11  12  13  14
  15  16  17  18  19
  20  21   *  22  23

BFS Path: ['right', 'right']
DFS Path: ['left', 'left', 'up', 'right', 'right', 'right', 'right', 'down', 'left', 'left', 'left', 'left', 'up', 'right', 'right', 'right', 'right', 'down', 'left', 'left', 'left', 'left', 'up', 'right', 'right', 'right', 'right', 'down', 'left', 'left', 'left', 'left', 'up', 'right', 'right', 'right', 'right', 'down', 'left', 'left', 'left', 'left', 'up', 'right', 'right', 'right', 'right', 'down', 'left', 'left', 'left', 'left', 'up', 'right', 'right', 'right', 'right', 'down', 'left', 'left', 'left', 'left', 'up', 'right', 'right', 'right', 'right', 'down', 'left', 'left', 'left', 'left', 'up', 'right', 'right', 'right', 'right', 'down', 'left', 'left', 'left', 'left', 'up', 'right', 'right', 'right', 'right', '